# Testing the constant-time case


## Import useful packages

In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale


ModuleNotFoundError: No module named 'numpy.testing.nosetester'

#### Define image paths

In [ ]:
image_dir='/w251-project/stars/images'

## Define functions to load the data


In [ ]:
def load_image(image_path, dim=(224,224)):
  """
  Loads a single image as a Numpy array and resizes it as
  desired.  The default dimensions are consistent with
  those expected by the VGG models.  

  Args:
    image_path: str pointing to the file

    dim: Two-element tuple giving the desired height
         and width of the processed image

  Returns:
    image:  A single-channel Numpy array
    """
  image = cv2.imread(image_path, 0)
  image = cv2.resize(image, dim)#, interpolation = cv2.INTER_AREA)
  return image

In [ ]:
def build_input(image_dir):
  """
  Loads all of the images into a single numpy array.
  Assumes that there are 101 equally-spaced images
  spanning lattitudes from 35N to 45N.  

  Args:
    image_dir: str giving name of the image directory

  Returns:
    X:  A 3-dimensional numpy array containing the
        images. Image height and width are set by
        `load_images` and default to 224 x 224.
    
    y:  A 1-dimensional numpy array of target lattitudes.
    """
  X = []
  print('about to do listdir') 
  files = os.listdir(image_dir)
  print('finished listdir')
  print(files)
  for file in files:
    if file[-4:] == '.png':
      image_path = os.path.join(image_dir, file)
      image = load_image(image_path)
      X.append(image)
  return (np.array(X) / 255)

In [2]:
def build_labels(image_dir):
  files = os.listdir(image_dir)
  y = []
  for file in files:
    if file[-4:] == '.png':
      file_split = file.split('+')
      lat = float(file_split[0])
      long = float(file_split[1])
      y.append((lat, long))
  return np.array(y)

In [3]:
file_name='43.20383241355868+-68.24170969718236+2020-05-25T00:59:02.png'
sample_file=os.path.join(image_dir,file_name)
print (sample_file)
sample_image = load_image(sample_file)
plt.imshow(sample_image)
print(sample_image.shape)

NameError: name 'image_dir' is not defined

## Prepare the data

### Load the images

In [7]:
print ('test')

test


In [ ]:
print('about to call build_input')
X = build_input(image_dir)

In [11]:
X.shape

AttributeError: 'list' object has no attribute 'shape'

In [9]:
y = build_labels(image_dir)

In [10]:
y.shape

(334, 2)


Standardize the values

In [ ]:
y[:,0] = scale(y[:,0])
y[:,1] = scale(y[:,1])

In [ ]:
print(y[:,0].mean(), y[:,0].std())
print(y[:,1].mean(), y[:,1].std())

### Split into test and training sets

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

## The model

We can probably prune this back a bit.  This is simply a DNN with single layer output suitable for regression.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224,224)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2)
])

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00001)

In [ ]:
model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

`callbacks` can be used to tell the model when to stop training.  In this case, I'm having it stop if the mean squared error hasn't improved for 20 epochs.

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='mean_squared_error', patience=30)

Fit the model.

In [ ]:
history = model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val),
                    verbose=2, epochs=1000, callbacks=[callback])

### Evaluate model performance

In [ ]:
history_df = pd.DataFrame(history.history)
ax = history_df.plot(y=['loss', 'val_loss'],
                xticks=np.linspace(0, len(history_df), 6))
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.set_title('Evolution of Mean Squared Error')
plt.show()


In [ ]:
y_hat = model.predict(X_val)

In [ ]:
plt.scatter(y_val[:,0], y_hat[:,0])
plt.title('Predicted lat vs. True lat')
plt.xlabel('True lat')
plt.ylabel('Predicted lat')
plt.show()

In [ ]:
plt.scatter(y_val[:,1], y_hat[:,1])
plt.title('Predicted long vs. True long')
plt.xlabel('True long')
plt.ylabel('Predicted long')
plt.show()